TODO: Split data in train, validation, test 

In [ ]:
%load_ext autoreload
%autoreload 2
from utils import FusionDataset
from models import Forward, Posterior, Prior, Decoder
from torch.utils.data import DataLoader
from train import run
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras

In [ ]:
# Instantiate models
forward_t = Forward()
forward_tplus1 = Forward()
prior = Prior()
posterior = Posterior()
decoder = Decoder()

# Instantiate optimizer
opt = keras.optimizers.AdamW()

# Get data
fusion_dataset = DataLoader(FusionDataset(), batch_size=16)

# Run training
run(fusion_dataset, forward_t, forward_tplus1, prior, posterior, decoder, opt, 10)